In [ ]:
import pandas as pd
import os.path as osp 
import numpy as np
import matplotlib.pyplot as plt
import os
import h5py
import json

In [ ]:
root_dir = "E:\hiwi\RADIal"
save_file = "E:\hiwi\data_procesing"
data_dir = "E:\\hiwi\\RADIal"
pcl_dir = "E:\\hiwi\\RADIal\\radar_PCL"
print(pcl_dir)
labels = pd.read_csv(os.path.join(root_dir,'labels.csv')).to_numpy()

In [ ]:
# Gather each input entries by their sample id
unique_ids = np.unique(labels[:,0])
label_dict = {}
for i,ids in enumerate(unique_ids):
    sample_ids = np.where(labels[:,0]==ids)[0]
    label_dict[ids]=sample_ids
sample_keys = list(label_dict.keys())

In [ ]:
index =np.argwhere(labels=='RECORD@2020-11-22_11.24.02')
print(index)

In [ ]:
#0,numsample 10,11,12,13 radar_R_m,radar_A_deg,radar_D_mps,radar_P_db 

#label_data = labels[:,[0,10,11,12,13]]
#np.save(osp.join(save_file,'lable.npy'), label_data)
labels = np.load(os.path.join(save_file,'lable.npy'),allow_pickle=True)
print(labels.shape)
"""
eps = np.finfo(float).eps
print(max(labels[:,4]))
print(min(labels[:,4]))
print(np.log10(max(labels[:,4])))
print(np.log10(min(abs(labels[:,4]))))
"""
#RAD power
print(labels[0,:])
RANGE_Resolution = np.around((103/512),2)
Azimuth_Resolution =np.around((180/751),1)
print(Azimuth_Resolution)
Doppler_index =6
Range_index= 3
Azimuth_index =8
a=[1,2,3,4,5,6]
print(a[1:4])

In [ ]:
"""
filename = os.path.join(root_dir,'radar_PCL','pcl_000018.npy')


data = np.load(filename,allow_pickle=True)
print(data.shape)
# range
#print(data[0,0])
#azimuth
#print(data[1,0])
#elevation
#print(data[2,0])
#power
#print(max(data[3,:]))
#doppler
#print(data[4,0])
#x
print(data[5,0:20])
#y
#print(max(data[6,:]))

"""

In [ ]:
number_sample = labels[:,0]
#radar_name = os.path.join(self.root_dir,'radar_FFT',"fft_{:06d}.npz".format(sample_id))
print(number_sample[0])
radar_fft = os.path.join(root_dir,'radar_FFT',"fft_{:06d}.npy".format(number_sample[0]))
print(radar_fft)

In [ ]:
def get_RA(RD_spectrums):

    doppler_indexes = []
    AoA_mat = np.load('CalibrationTable.npy',allow_pickle=True).item()
    window = AoA_mat['H'][0]
    numChirps = 256
    numReducedDoppler = 16
    numChirpsPerLoop = 16
    CalibMat=AoA_mat['Signal'][...,5]
    dividend_constant_arr = np.arange(0,numReducedDoppler*numChirpsPerLoop ,numReducedDoppler)
    for doppler_bin in range(numChirps):
        DopplerBinSeq = np.remainder(doppler_bin+ dividend_constant_arr,numChirps)
        DopplerBinSeq = np.concatenate([[DopplerBinSeq[0]],DopplerBinSeq[5:]])
        doppler_indexes.append(DopplerBinSeq)

    MIMO_Spectrum = RD_spectrums[:,doppler_indexes,:].reshape(RD_spectrums.shape[0]*RD_spectrums.shape[1],-1)


    # Multiply with Hamming window to reduce side lobes
    MIMO_Spectrum = np.multiply(MIMO_Spectrum,window)

    Azimuth_spec = np.abs(CalibMat@MIMO_Spectrum.transpose())
    Azimuth_spec = Azimuth_spec.reshape(AoA_mat['Signal'].shape[0],RD_spectrums.shape[0],RD_spectrums.shape[1])
    #(751, 512, 256) A,R,D
    #RAD
    #print(Azimuth_spec.shape)
    Azimuth_spec = np.transpose(Azimuth_spec, (1, 0, 2))
    #RA_map = np.sum(np.abs(Azimuth_spec),axis=2)
    
    return Azimuth_spec

In [ ]:
class RADIal():
    
    def __init__(self, root_dir,label_file):
        self.root_dir = root_dir
        self.label_file = label_file
        self.labels = np.load(os.path.join(label_file,'lable.npy'),allow_pickle=True)
    
    def to_radar_cube(self):
        number_sample = self.labels[:,0]
        Range_fft = self.labels[:,1]
        Azimuth_fft = self.labels[:,2]
        Doppler_index = self.labels[:,3]
        Range_Resolution = 103/512
        Azimuth_Resolution =180/751
        
        for index in range(number_sample.shape[0]):
            Range_fft = self.labels[index,1]
            Azimuth_fft = self.labels[index,2]
            Doppler_index = int(self.labels[index,3])
            Range_index = int(Range_fft/Range_Resolution)
            
            Azimuth_index = int(Azimuth_fft+90/Azimuth_Resolution)
                
            
            
                
            
            #print(Doppler_index)
            #print(Range_index)
            #print(Azimuth_index)
            #  Doppler_index <= 255-17
            if Doppler_index >=15 and Doppler_index <= 238:
                
                radar_fft_path= os.path.join(self.root_dir,'radar_FFT',"fft_{:06d}.npy".format(number_sample[index]))
                radar_fft = np.load(radar_fft_path,allow_pickle=True)
                radar_fft = np.concatenate([radar_fft.real,radar_fft.imag],axis=2)
                radar_fft = radar_fft[...,::2]+1j*radar_fft[...,1::2]
                radar_cube = get_RA(radar_fft)
                
                radar_cube = radar_cube[Range_index-2:Range_index+3,Azimuth_index-2:Azimuth_index+3,Doppler_index-15:Doppler_index+17]
                
                
                save_file = os.path.join(self.label_file,"radar_cube_{:06d}.hdf5".format(number_sample[index]))
                f = h5py.File(save_file,'a')
                d1 = f.create_dataset(str(index),data=radar_cube)
            elif Doppler_index <15:
                radar_fft_path= os.path.join(self.root_dir,'radar_FFT',"fft_{:06d}.npy".format(number_sample[index]))
                radar_fft = np.load(radar_fft_path,allow_pickle=True)
                radar_fft = np.concatenate([radar_fft.real,radar_fft.imag],axis=2)
                radar_fft = radar_fft[...,::2]+1j*radar_fft[...,1::2]
                radar_cube = get_RA(radar_fft)
                
                radar_cube_1 = radar_cube[Range_index-2:Range_index+3,Azimuth_index-2:Azimuth_index+3,Doppler_index-15:]
                radar_cube_2 = radar_cube[Range_index-2:Range_index+3,Azimuth_index-2:Azimuth_index+3,0:Doppler_index+17]
                radar_cube = np.concatenate((radar_cube_1, radar_cube_2), axis=2)
                
                save_file = os.path.join(self.label_file,"radar_cube_{:06d}.hdf5".format(number_sample[index]))
                f = h5py.File(save_file,'a')
                d1 = f.create_dataset(str(index),data=radar_cube)
            else:
                radar_fft_path= os.path.join(self.root_dir,'radar_FFT',"fft_{:06d}.npy".format(number_sample[index]))
                radar_fft = np.load(radar_fft_path,allow_pickle=True)
                radar_fft = np.concatenate([radar_fft.real,radar_fft.imag],axis=2)
                radar_fft = radar_fft[...,::2]+1j*radar_fft[...,1::2]
                radar_cube = get_RA(radar_fft)
                
                radar_cube_1 = radar_cube[Range_index-2:Range_index+3,Azimuth_index-2:Azimuth_index+3,Doppler_index-15:]
                radar_cube_2 = radar_cube[Range_index-2:Range_index+3,Azimuth_index-2:Azimuth_index+3,0:17-(255-Doppler_index)]
                radar_cube = np.concatenate((radar_cube_1, radar_cube_2), axis=2)
                
                save_file = os.path.join(self.label_file,"radar_cube_{:06d}.hdf5".format(number_sample[index]))
                f = h5py.File(save_file,'a')
                d1 = f.create_dataset(str(index),data=radar_cube)
            
            if index%10==0:
                print(f.keys())
                
            if index%10==0:
                print(index)
            f.close()
                

                    
                    
                
                
            


In [ ]:
radar_data = RADIal(data_dir,save_file)
radar_data.to_radar_cube()



In [ ]:
filename =os.path.join(save_file,"radar_cube_000018.hdf5")
f2.close()
f2 = h5py.File(filename,'r')
print(f2[str(0)][:])
for key in f2.keys():
    print(key)
#print(f2['0'].shape)
f2.close()

In [ ]:
filename = os.path.join(root_dir,'radar_PCL','pcl_000018.npy')
data = np.load(filename,allow_pickle=True)[0:-4,:]
radar_r_data =data[0,:]

radar_a_data =data[1,:]
eva = data[2,:]
radar_p_data =data[3,:]
radar_d_data =data[4,:]

In [ ]:
#print(radar_d_data[10:20])
#print(radar_p_data[0:10])
#range 103m,Azimuth 108 ,
print(radar_d_data.shape)
print(max(radar_d_data))
print(radar_r_data[470:480])
print(radar_d_data[470:480])
print(radar_a_data[470:480])

In [ ]:
filename = os.path.join(root_dir,'radar_FFT','fft_000018.npy')
fft_data = np.load(filename,allow_pickle=True)
print(fft_data.shape)

In [ ]:
radar_FFT = np.concatenate([fft_data.real,fft_data.imag],axis=2)
print(radar_FFT.shape)

In [ ]:
radar_FFT = radar_FFT[...,::2]+1j*radar_FFT[...,1::2]
print(radar_FFT.shape)

In [ ]:
power_spectrum = np.sum(np.abs(radar_FFT),axis=2)
print(power_spectrum.shape)

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(np.log10(power_spectrum))

In [ ]:
AoA_mat = np.load('CalibrationTable.npy',allow_pickle=True).item()
window = AoA_mat['H'][0]
numChirps = 256
numReducedDoppler = 16
numChirpsPerLoop = 16
CalibMat=AoA_mat['Signal'][...,5]
dividend_constant_arr = np.arange(0,numReducedDoppler*numChirpsPerLoop ,numReducedDoppler)

In [ ]:
def get_RA(RD_spectrums,window,numChirps,dividend_constant_arr,CalibMat):

    doppler_indexes = []
    for doppler_bin in range(numChirps):
        DopplerBinSeq = np.remainder(doppler_bin+ dividend_constant_arr,numChirps)
        DopplerBinSeq = np.concatenate([[DopplerBinSeq[0]],DopplerBinSeq[5:]])
        doppler_indexes.append(DopplerBinSeq)

    MIMO_Spectrum = RD_spectrums[:,doppler_indexes,:].reshape(RD_spectrums.shape[0]*RD_spectrums.shape[1],-1)


    # Multiply with Hamming window to reduce side lobes
    MIMO_Spectrum = np.multiply(MIMO_Spectrum,window)

    Azimuth_spec = np.abs(CalibMat@MIMO_Spectrum.transpose())
    Azimuth_spec = Azimuth_spec.reshape(AoA_mat['Signal'].shape[0],RD_spectrums.shape[0],RD_spectrums.shape[1])
    #radar cube

    #RA_map = np.sum(np.abs(Azimuth_spec),axis=2)
    #(751, 512, 256) A,R,D
    return Azimuth_spec

In [ ]:
ra = get_RA(radar_FFT,window,numChirps,dividend_constant_arr,CalibMat)




In [ ]:
print("A,R,D")
print(ra.shape)
#ra_rad 最终的radar cube RAD
ra_rad = np.transpose(ra, (1, 0, 2))
print("R,A,D")
print(ra_rad.shape)

In [ ]:
x = np.arange(28).reshape((2,2,7))
print(x)
x = np.transpose(x, (1, 0, 2))
print(x)

In [ ]:
plt.imshow(20*np.log10(np.abs(ra)))